In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from cgmap.mapping.mapper import Mapper

args_dict = {
    'mapping': 'martini3',
    'input': '/storage_common/angiod/chignolin_unfolded/chignolin.gro',
    'inputtraj': '/storage_common/angiod/chignolin_unfolded/chignolin_forces_unfolded_fitted_whole.trr',
    'traj_format': 'trr',
    'trajslice': ':100',
    'selection': 'protein',
    'output': 'pro.pdb',
    'outputtraj': 'xtc',
    'isatomistic': True,
}

mapping = Mapper(args_dict=args_dict)

# --- Option 1 --- #
# Map only the ith file in the list of files (and optionally trajectories)

mapping.map(index=0)
# mapping.save()
# mapping.save_atomistic()
mapping.save_npz(filename='data.npz', pos_unit='Angstrom', force_unit='kcal/(mol*Angstrom)')

# --- Option 2 --- #
# Map all the files one after the other

# for i, m in enumerate(mapping()):
#     m.save()
#     m.save_atomistic()
#     m.save_npz(filename=f'data_{i}.npz', pos_unit='Angstrom', force_unit='kcal/(mol*Angstrom)')